In [1]:
import importlib
import sys
from tqdm import tqdm
import  os
import pickle
import glob
from datetime import datetime
import matplotlib.pyplot as plt
# Add the directories to the sys.path
dir_path = "C://Users//jony//Programming//Python//Anubis//anubis//" # insert your directory path
sys.path.append(dir_path + "Osiris//processing//python")
sys.path.append(dir_path + "tools")

import Analysis_tools as ATools
import Atlas_tools as AtTools
import proAnubis_Analysis_Tools
import Reconstruction_tools as RTools
import numpy as np
import mplhep as hep
import Timing_tools as TTools
import pickle
import overview
import rawFileReader
import Visual_tools as VTools
hep.style.use([hep.style.ATLAS])

dir = "C://Users//jony//Programming//Python//Anubis//anubis//data//"

# Specify the directory


In [2]:
importlib.reload(overview)
file_name = "proAnubis_240815_1759.raw"
chunks, times, fReader = overview.get_chunks(file_name, max_process_event=10_000_000, saves = False)

Initial time 2024-08-15 16:59:49.312501 1723737589.312501


Processing Chunks proAnubis_240815_1759.raw:  91%|█████████▏| 91444/100000 [54:24<01:09, 123.64Chunks/s]   

In [3]:
#find events with multiple clusters
importlib.reload(RTools)
importlib.reload(proAnubis_Analysis_Tools)
importlib.reload(rawFileReader)

file_name = "proAnubis_240815_1759.raw"

fReader = rawFileReader.fileReader(dir+file_name)
tol = [i/10 for i in range(100)]
possible_reconstructions = [0 for rpc in range(6)]
successful_reconstructions = [[0 for _ in tol] for rpc in range(6)]
chi2 = []

interesting_events = []

idonotstop = True
total_events = 0
while idonotstop:
   chunks, times, fReader = overview.get_chunks(file_name, max_process_event=1_000_000, fReader = fReader, saves = False)
   print("Total Events:", total_events)
   print("Total reconstructed events:", possible_reconstructions)
   reconstructor = proAnubis_Analysis_Tools.Reconstructor(chunks[0], 0)
   for processedEvents, event_chunk in enumerate(chunks[1:]):
      reconstructor.update_event(event_chunk, processedEvents)
      # populate_hits turns TDC bit wise information into their corresponding strips
      reconstructor.populate_hits()
      # This is optionnal, and requires the residual of eta and phi
         # make_cluster does temporal and spatial coincidence between the stips, and reconstruction is done
      clusters = reconstructor.make_cluster()
      chi2_region = (0, 100)
      for evt_num, cluster in enumerate(clusters):
         for rpc in range(6):
            E_recon = RTools.reconstruct_timed_Chi2_ByRPC(cluster, 3, rpc) #why is it excluded?
            if E_recon and len(E_recon) > 1:
                  interesting_events.append(event_chunk[cluster[0]-processedEvents])
                  VTools.event_3d_plot(event_chunk[cluster[0]-processedEvents], processedEvents*100+10*evt_num)
                  track = E_recon[1]
                  if len(track[2]) >= 5:
                     if track[4] > chi2_region[0] and track[4] < chi2_region[1]:
                           chi2.append(track[4])
                           # self.event_of_interest.append(track)
                           # Adding this check to see if other 5 RPCs are in reconstructed event.
                           # This is necessary to ensure the reconstructed path is accurate.

                           muon_coords = RTools.does_muon_hit_RPC(track[0], track[1], rpc)
                           if muon_coords:
                              possible_reconstructions[rpc] += 1
                              for idx, t in enumerate(tol):
                                 check = RTools.does_RPC_detect_muon(muon_coords, track[7], t)
                                 if check:
                                    successful_reconstructions[rpc][idx] += 1
      
      total_events += 1_000_000
      with open(f"data//double//interesting_events{total_events//1_000_000}.pkl", "wb") as f:
         pickle.dump(interesting_events, f)
                                      
      """
      for evt_num, cluster in enumerate(clusters):
         repeated = 0
         for rpc in range(6):
            if len(cluster[2][rpc][0]) == 2 and len(cluster[2][rpc][1]) == 2: #phi and eta
               repeated += 1
         #tracks = RTools.reconstruct_timed_Chi2_ByRPC(cluster, 3, -1)
         if len(tracks) > 1
            #heatmap
            hits = []
            for rpc in range(6):
               hits.extend([phi for phi in cluster[2][rpc][0]])
               hits.extend([eta for eta in cluster[2][rpc][1]])
            hits = [h[0] for h in hits]
            heatMaps = [np.zeros((32,64)) for rpc in range(6)]
            for hit in hits:
               if hit[3]:
                  heatMaps[hit[0]][hit[1],:] += np.ones(64)
               else:
                  heatMaps[hit[0]][:,hit[1]] += np.ones(32)
            #reconstruction

            print("Number of tracks:", len(tracks))
            for track in tracks:
               centroid, direction, opt_coords, _, chi2, dT, _, _ = track
               for rpc, point in enumerate(opt_coords):
                  try:
                     heatMaps[rpc][31 - int(point[1]/ 2.9844)][int(point[0]/ 2.7625)] += 3
                  except:
                     print((int(point[0]/ 2.7625), int(point[1]/ 2.9844)))

            for rpc in range(6):
                  plt.imshow(heatMaps[rpc], interpolation='nearest')
                  plt.title(f'Event {evt_num} RPC {rpc}')
                  plt.savefig(f'video/images_video/double{processedEvents*100+10*evt_num+rpc}.png')
         """
      #print("Clust")

Initial time 2024-08-15 16:59:49.312501 1723737589.312501


Processing Chunks proAnubis_240815_1759.raw: 100%|██████████| 10000/10000 [01:45<00:00, 94.49Chunks/s]


Ending time: 2024-08-15 17:03:04.904445
Number of chunks: 10000
Total Events: 0
Total reconstructed events: [0, 0, 0, 0, 0, 0]


C:\/Users//jony//Programming//Python//Anubis//anubis//tools\Visual_tools.py:102: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`). Consider using `matplotlib.pyplot.close()`.
  fig = plt.figure(figsize=(10, 10))
C:\/Users//jony//Programming//Python//Anubis//anubis//tools\Reconstruction_tools.py:362: RuntimeWarning: divide by zero encountered in scalar divide
  t = (z-z_0)/d[2]
C:\/Users//jony//Programming//Python//Anubis//anubis//tools\Reconstruction_tools.py:367: RuntimeWarning: invalid value encountered in scalar multiply
  y_traj = centroid[1] + t*d[1]


Initial time 2024-08-15 17:03:04.921412 1723737784.921412


Processing Chunks proAnubis_240815_1759.raw: 100%|██████████| 10000/10000 [01:46<00:00, 93.86Chunks/s]


Ending time: 2024-08-15 17:06:18.967902
Number of chunks: 10000
Total Events: 9999000000
Total reconstructed events: [23, 13, 9, 27, 19, 14]
Initial time 2024-08-15 17:06:18.987834 1723737978.987834


Processing Chunks proAnubis_240815_1759.raw: 100%|██████████| 10000/10000 [01:50<00:00, 90.48Chunks/s]


Ending time: 2024-08-15 17:09:32.379433
Number of chunks: 10000
Total Events: 19998000000
Total reconstructed events: [36, 21, 17, 52, 33, 23]


: 

In [19]:
importlib.reload(VTools)
file_path = "data//interesting_events.pkl"
with open(file_path, 'rb') as f:
    data = pickle.load(f)
    print(len(data))
interesting_events = data
for event in interesting_events[10:11]:
   VTools.event_3d_plot(event, 0)

260


In [11]:
reconstructor = proAnubis_Analysis_Tools.Reconstructor(interesting_events, 0)
# populate_hits turns TDC bit wise information into their corresponding strips
reconstructor.populate_hits()
# This is optionnal, and requires the residual of eta and phi
    # make_cluster does temporal and spatial coincidence between the stips, and reconstruction is done
clusters = reconstructor.make_cluster()

chi2 = []
chi2_region = (0, 100)
for evt_num, cluster in enumerate(clusters):
    tracks = RTools.reconstruct_timed_Chi2_ByRPC(cluster, 3, -1)
    if tracks and len(tracks) > 1:
        chi2.append(tracks[1][4])
    print(evt_num)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181


KeyboardInterrupt: 

In [12]:
plt.figure(figsize=(10, 6))
max_eff = []
for RPC in range(6):
    if possible_reconstructions[RPC] == 0:
        efficiency = [0 for x in successful_reconstructions[RPC]]
    else:
        efficiency = [x / possible_reconstructions[RPC] for x in successful_reconstructions[RPC]]
    max_eff.append(efficiency[-1])
    plt.plot(tol, efficiency, label=f'RPC {RPC}')

plt.xlabel('Tolerance')
plt.ylabel('Efficiency')
plt.title('The efficiency of the second track reconstruction')
plt.legend()
plt.grid(True)
plt.show()
plt.close()
print("Efficiency Done")
print("Possible reconstructions", reconstructor.possible_reconstructions)
print(max_eff)

Efficiency Done
Possible reconstructions [0, 0, 0, 0, 0, 0]
[0.4318181818181818, 0.6, 0.6818181818181818, 0.4406779661016949, 0.35, 0.6071428571428571]


In [18]:
#plot chi2 distribution
import scipy.special as sp
k = np.mean(chi2)
plt.hist(chi2, bins = int(np.sqrt(8*len(chi2))), density = True)
x = np.array([i for i in range(0, 10)])
print(sp.gamma(k))
plt.plot(x, 2.4*x**(k/2-1)*np.exp(-x/2)/((2**(k/2))*sp.gamma(k/2)), label = f"Idealised χ2({round(k,2)}) distribution")#pdf of chi2
plt.title('χ2 distribution of the second track')
plt.xlabel('Chi2')
plt.legend()
plt.ylabel('Frequency')
plt.show()
print(chi2)

3.549413719172803
[np.float64(2.923268450788971), np.float64(1.5455626187848202), np.float64(1.3824512933971917), np.float64(0.2870659845529911), np.float64(6.057028098867192), np.float64(1.1170741127596033), np.float64(0.47158490850782714), np.float64(1.4008653273049967), np.float64(6.473761420102736), np.float64(1.6294728836774055), np.float64(1.5255178606337314), np.float64(0.4398499126206219), np.float64(3.4205517579528855), np.float64(2.100629585341305), np.float64(3.8330354225996115), np.float64(1.600927888787882), np.float64(4.449677296942161), np.float64(4.449677296942161), np.float64(4.449677296942161), np.float64(4.449677296942161), np.float64(4.449677296942161), np.float64(4.449677296942161), np.float64(4.6383741165254815), np.float64(4.6383741165254815), np.float64(4.6383741165254815), np.float64(4.6383741165254815), np.float64(4.6383741165254815), np.float64(4.6383741165254815), np.float64(5.698472209406913), np.float64(5.698472209406913), np.float64(5.698472209406913), np